In [1]:
pip install sentence-transformers faiss-cpu numpy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np
import faiss

# Generate embeddings
embeddings = model.encode(titles, convert_to_numpy=True, show_progress_bar=True)

# Create Faiss index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

np.save("title_embeddings.npy", embeddings)
faiss.write_index(index, "faiss_index_ms_marco.index")

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

## cosine similarity is used here
### Cosine similarity	Cosine of the angle between two vectors (after normalizing)	Higher = more similar (smaller angle)
### Cosine similarity, by normalizing vectors, ignores magnitude and focuses purely on “direction” (semantic content).

## metadata.json open

In [ ]:
import json

with open("metadata.json", "r") as f:
    metadata = json.load(f)

titles = [entry["title"] for entry in metadata]


## sentence transformers


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("msmarco-distilbert-base-v4")


## load embeddings

In [ ]:
embeddings = np.load("title_embeddings.npy")
index      = faiss.read_index("faiss_index_ms_marco.index")
# reload model & metadata, then define search()…


## search
## threshold applied

In [30]:
import faiss
import numpy as np

# — After you’ve computed `embeddings` as before —

# 1. Normalize embeddings to unit length
faiss.normalize_L2(embeddings)

# 2. Build an inner-product index instead of L2
dim   = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(embeddings)

# 3. When you search, normalize the query too:
def search_cosine(query, top_k=3, threshold=0.5):
    # a) embed
    q_emb = model.encode([query], convert_to_numpy=True)
    # b) normalize
    faiss.normalize_L2(q_emb)
    # c) search inner product
    distances, indices = index.search(q_emb, top_k)
    
    results = [
        {
            "title":   metadata[i]["title"],
            "chapter": metadata[i]["chapter"],
            "cosine_sim": float(distances[0][j])
        }
        for j, i in enumerate(indices[0])
        if float(distances[0][j]) >= threshold
    ]
    
    if not results:
        return "Content not found in knowledge base."
    
    return results

# 4. Try it out:
results = search_cosine(" newtons law", top_k=3, threshold=0.5)
if isinstance(results, str):
    print(results)
else:
    for r in results:
        print(r)


Content not found in knowledge base.
